<a href="https://colab.research.google.com/github/agniji/VisionSystems/blob/main/Production_Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Install Dependencies

In [ ]:
!pip install opencv-python torch torchvision numpy PySpin


##Python Script for Real-Time Inference (production_inference.py)

In [ ]:
import cv2
import numpy as np
import torch
import PySpin

# Load the trained model (TorchScript format for portability)
MODEL_PATH = "model.pt"
model = torch.jit.load(MODEL_PATH)
model.eval()

# Initialize FLIR camera
def initialize_camera():
    system = PySpin.System.GetInstance()
    cam_list = system.GetCameras()

    if cam_list.GetSize() == 0:
        print("No cameras detected!")
        system.ReleaseInstance()
        return None, None

    camera = cam_list.GetByIndex(0)
    camera.Init()
    return camera, system

# Preprocess frame for model input
def preprocess_frame(image):
    image = cv2.resize(image, (224, 224))  # Adjust to your model's input size
    image = np.transpose(image, (2, 0, 1))  # Convert HWC to CHW
    image = image / 255.0  # Normalize
    image = torch.tensor(image, dtype=torch.float32).unsqueeze(0)  # Add batch dim
    return image

# Main loop to capture and infer from camera feed
def stream_and_infer():
    camera, system = initialize_camera()
    if camera is None:
        return

    print("Starting real-time inference...")

    try:
        while True:
            # Capture image
            image = camera.GetNextImage()
            if image.IsIncomplete():
                print("Image capture incomplete. Skipping frame.")
                continue

            # Convert to OpenCV format
            img_data = image.GetNDArray()
            img_data = cv2.cvtColor(img_data, cv2.COLOR_BAYER_BG2RGB)  # Convert Bayer to RGB
            image.Release()

            # Preprocess and run inference
            input_tensor = preprocess_frame(img_data)
            with torch.no_grad():
                output = model(input_tensor)

            # Process model output (Modify as per your use case)
            pred_label = torch.argmax(output).item()
            confidence = torch.nn.functional.softmax(output, dim=1)[0, pred_label].item()
            label_text = f"Pred: {pred_label} ({confidence:.2f})"

            # Display results
            cv2.putText(img_data, label_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX,
                        1, (0, 255, 0), 2)
            cv2.imshow("Blackfly S - Real-Time Inference", img_data)

            # Exit on 'q' key
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    finally:
        camera.DeInit()
        system.ReleaseInstance()
        cv2.destroyAllWindows()
        print("Camera released, resources cleaned.")

# Run the script
if __name__ == "__main__":
    stream_and_infer()


##Package it as an executable

In [ ]:
!pyinstaller --onefile production_inference.py